# Housing Rental Analysis for San Francisco
This project uses interactive charts and maps to analyze changes in the San Francisco real estate market between 2012 and 2016. The analysis will be performed within the context of the following scenario:

>Assume that you’re an analyst at a proptech company that wants to offer an instant, one-click service for people to buy properties and then rent them. The company wants to have a trial of this offering in the San Francisco real-estate market. If the service proves popular, they can then expand to other markets. Your job is to use your data visualization skills, including aggregation, interactive visualizations, and geospatial analysis, to find properties in the San Francisco market that are viable investment opportunities.

In [94]:
# Import the required libraries and dependencies
import pandas as pd
import hvplot.pandas
from pathlib import Path
from bokeh.models.formatters import NumeralTickFormatter
formatter = NumeralTickFormatter(format="0,0")

## Import the data 

In [95]:
# Using the read_csv function and Path module, create a DataFrame 
# by importing the sfo_neighborhoods_census_data.csv file from the Resources folder
sfo_data_df = pd.read_csv(
    Path('./Resources/sfo_neighborhoods_census_data.csv')
)

# Review the first and last five rows of the DataFrame
display(sfo_data_df.head())
display(sfo_data_df.tail())

,year,neighborhood,sale_price_sqr_foot,housing_units,gross_rent
0,2010,Alamo Square,291.182945,372560,1239
1,2010,Anza Vista,267.932583,372560,1239
2,2010,Bayview,170.098665,372560,1239
3,2010,Buena Vista Park,347.394919,372560,1239
4,2010,Central Richmond,319.027623,372560,1239


,year,neighborhood,sale_price_sqr_foot,housing_units,gross_rent
392,2016,Telegraph Hill,903.049771,384242,4390
393,2016,Twin Peaks,970.085470,384242,4390
394,2016,Van Ness/ Civic Center,552.602567,384242,4390
395,2016,Visitacion Valley,328.319007,384242,4390
396,2016,Westwood Park,631.195426,384242,4390


---

## Calculate and Plot the Housing Units per Year

### Use the `groupby` function to group the data by year. Aggregate the results by the `mean` of the groups.

In [96]:
# Create a numerical aggregation that groups the data by the year and then averages the results.
housing_units_by_year = sfo_data_df.groupby('year').mean()

# Review the DataFrame
housing_units_by_year

,sale_price_sqr_foot,housing_units,gross_rent
year,,,
2010,369.344353,372560.0,1239.0
2011,341.903429,374507.0,1530.0
2012,399.389968,376454.0,2324.0
2013,483.600304,378401.0,2971.0
2014,556.277273,380348.0,3528.0
2015,632.540352,382295.0,3739.0
2016,697.643709,384242.0,4390.0


### Use the `hvplot` function to plot the `housing_units_by_year` DataFrame as a bar chart.

In [97]:
# Create a visual aggregation explore the housing units by year
housing_units_by_year.hvplot.bar(
    x='year',
    y='housing_units',
    xlabel='Year',
    ylabel='Number of Housing Units',
    title='Number of Housing Units in San Francisco by Year',
    color='lightpink',
    hover_color='palevioletred'
).opts(
    yformatter=formatter,
    ylim=(365000, 385000)
)

:Bars   [year]   (housing_units)

### Question: What is the overall trend in housing_units over the period being analyzed?
Overall, it appears that the number of housing units increased by around 2000 units each year.

---

## Calculate and Plot the Average Sale Prices per Square Foot

Group the data by year, and then average the results.

In [98]:
# Create a numerical aggregation by grouping the data by year and averaging the results
prices_square_foot_by_year = sfo_data_df.groupby('year').mean()

# Review the resulting DataFrame
prices_square_foot_by_year.sort_values('gross_rent')

,sale_price_sqr_foot,housing_units,gross_rent
year,,,
2010,369.344353,372560.0,1239.0
2011,341.903429,374507.0,1530.0
2012,399.389968,376454.0,2324.0
2013,483.600304,378401.0,2971.0
2014,556.277273,380348.0,3528.0
2015,632.540352,382295.0,3739.0
2016,697.643709,384242.0,4390.0


### Question: What is the lowest gross rent reported for the years included in the DataFrame?
The lowest average gross rent reported was 1239 dollars in 2010.

### Create a new DataFrame named `prices_square_foot_by_year` by filtering out the “housing_units” column. The new DataFrame should include the averages per year for only the sale price per square foot and the gross rent.

In [99]:
# Filter out the housing_units column, creating a new DataFrame 
# Keep only sale_price_sqr_foot and gross_rent averages per year
prices_square_foot_by_year = prices_square_foot_by_year.drop(columns='housing_units')

# Review the DataFrame
prices_square_foot_by_year

,sale_price_sqr_foot,gross_rent
year,,
2010,369.344353,1239.0
2011,341.903429,1530.0
2012,399.389968,2324.0
2013,483.600304,2971.0
2014,556.277273,3528.0
2015,632.540352,3739.0
2016,697.643709,4390.0


In [100]:
prices_square_foot_by_year.sort_values('sale_price_sqr_foot')

,sale_price_sqr_foot,gross_rent
year,,
2011,341.903429,1530.0
2010,369.344353,1239.0
2012,399.389968,2324.0
2013,483.600304,2971.0
2014,556.277273,3528.0
2015,632.540352,3739.0
2016,697.643709,4390.0


### Use hvPlot to plot the `prices_square_foot_by_year` DataFrame as a line plot.

In [101]:
# Plot prices_square_foot_by_year. 
# Inclued labels for the x- and y-axes, and a title.
prices_square_foot_by_year.hvplot.line(
    xlabel='Year',
    ylabel='Sale Price per Square Foot and Gross Rent',
    title='San Francisco Average Gross Rent and Sale Price/Square Foot by Year, 2010-2016',
)

:NdOverlay   [Variable]
   :Curve   [year]   (value)

### Questions:
#### Did any year experience a drop in the average sale price per square foot compared to the previous year?
Yes, in 2011 the average sale price per square foot was 28 dollars less than in 2010. Other than that, however, the average sale price per square foot has followed a steady upward trend.
#### If so, did the gross rent increase or decrease during that year?
Despite the minor decrease in average sale price between 2010 and 2011, the average gross rent price increased by 291 dollars that year.

---

## Compare the Average Sale Prices by Neighborhood

### Create a new DataFrame that groups the original DataFrame by year and neighborhood. Aggregate the results by the `mean` of the groups.

In [102]:
# Group by year and neighborhood and then create a new dataframe of the mean values
prices_by_year_by_neighborhood = sfo_data_df.groupby(['year','neighborhood']).mean()

# Review the DataFrame
prices_by_year_by_neighborhood

sale_price_sqr_foot  housing_units  gross_rent
year neighborhood                                                          
2010 Alamo Square                     291.182945       372560.0      1239.0
     Anza Vista                       267.932583       372560.0      1239.0
     Bayview                          170.098665       372560.0      1239.0
     Buena Vista Park                 347.394919       372560.0      1239.0
     Central Richmond                 319.027623       372560.0      1239.0
...                                          ...            ...         ...
2016 Telegraph Hill                   903.049771       384242.0      4390.0
     Twin Peaks                       970.085470       384242.0      4390.0
     Van Ness/ Civic Center           552.602567       384242.0      4390.0
     Visitacion Valley                328.319007       384242.0      4390.0
     Westwood Park                    631.195426       384242.0      4390.0

[397 rows x 3 columns]

### Filter out the “housing_units” column to create a DataFrame that includes only the `sale_price_sqr_foot` and `gross_rent` averages per year.

In [103]:
# Filter out the housing_units
prices_by_year_by_neighborhood = prices_by_year_by_neighborhood.drop(columns='housing_units')

# Review the first and last five rows of the DataFrame
display(prices_by_year_by_neighborhood.head())
display(prices_by_year_by_neighborhood.tail())

sale_price_sqr_foot  gross_rent
year neighborhood                                     
2010 Alamo Square               291.182945      1239.0
     Anza Vista                 267.932583      1239.0
     Bayview                    170.098665      1239.0
     Buena Vista Park           347.394919      1239.0
     Central Richmond           319.027623      1239.0

sale_price_sqr_foot  gross_rent
year neighborhood                                           
2016 Telegraph Hill                   903.049771      4390.0
     Twin Peaks                       970.085470      4390.0
     Van Ness/ Civic Center           552.602567      4390.0
     Visitacion Valley                328.319007      4390.0
     Westwood Park                    631.195426      4390.0

### Create an interactive line plot with hvPlot that visualizes both `sale_price_sqr_foot` and `gross_rent`. Set the x-axis parameter to the year (`x="year"`). Use the `groupby` parameter to create an interactive widget for `neighborhood`.

In [104]:
# Use hvplot to create an interactive line plot of the average price per square foot
# The plot should have a dropdown selector for the neighborhood
prices_by_year_by_neighborhood.hvplot.line(
    x='year',
    xlabel='Year',
    ylabel='Avg. Sale Price/Sqft, Avg. Gross Rent',
    groupby='neighborhood',
    title='Average Sale Price per Square Foot and Average Gross Rent - By Neighborhood'
)

:DynamicMap   [neighborhood]
   :NdOverlay   [Variable]
      :Curve   [year]   (value)

### Question: For the Anza Vista neighborhood, is the average sale price per square foot for 2016 more or less than the price that’s listed for 2012? 
For the Anza Vista neighborhood, the average sale price per square foot was 256 dollars less in 2016 than in 2012.

---

## Build an Interactive Neighborhood Map
Explore the geospatial relationships in the data by using interactive visualizations with hvPlot and GeoViews.

### Read the `neighborhood_coordinates.csv` file from the `Resources` folder into the notebook, and create a DataFrame named `neighborhood_locations_df`.

In [105]:
# Load neighborhoods coordinates data
neighborhood_locations_df = pd.read_csv(
    Path('./Resources/neighborhoods_coordinates.csv'),
    index_col='Neighborhood'
)

# Review the DataFrame
neighborhood_locations_df

,Lat,Lon
Neighborhood,,
Alamo Square,37.791012,-122.402100
Anza Vista,37.779598,-122.443451
Bayview,37.734670,-122.401060
Bayview Heights,37.728740,-122.410980
Bernal Heights,37.728630,-122.443050
...,...,...
West Portal,37.740260,-122.463880
Western Addition,37.792980,-122.435790
Westwood Highlands,37.734700,-122.456854


### Using the original `sfo_data_df` Dataframe, create a DataFrame named `all_neighborhood_info_df` that groups the data by neighborhood. Aggregate the results by the `mean` of the group.

In [106]:
# Calculate the mean values for each neighborhood
all_neighborhood_info_df = sfo_data_df.groupby('neighborhood').mean()

# Review the resulting DataFrame
all_neighborhood_info_df

,year,sale_price_sqr_foot,housing_units,gross_rent
neighborhood,,,,
Alamo Square,2013.000000,366.020712,378401.00,2817.285714
Anza Vista,2013.333333,373.382198,379050.00,3031.833333
Bayview,2012.000000,204.588623,376454.00,2318.400000
Bayview Heights,2015.000000,590.792839,382295.00,3739.000000
Bernal Heights,2013.500000,576.746488,379374.50,3080.333333
...,...,...,...,...
West Portal,2012.250000,498.488485,376940.75,2515.500000
Western Addition,2012.500000,307.562201,377427.50,2555.166667
Westwood Highlands,2012.000000,533.703935,376454.00,2250.500000


### Concatenate the `neighborhood_locations_df` DataFrame with the `all_neighborhood_info_df` DataFrame. 

In [107]:
# Using the Pandas `concat` function, join the 
# neighborhood_locations_df and the all_neighborhood_info_df DataFrame
# The axis of the concatenation is "columns".
# The concat function will automatially combine columns with
# identical information, while keeping the additional columns.
all_neighborhoods_df = pd.concat(
    [neighborhood_locations_df, all_neighborhood_info_df], 
    axis="columns",
    sort=False
)

# Review the resulting DataFrame
display(all_neighborhoods_df.head())
display(all_neighborhoods_df.tail())


,Lat,Lon,year,sale_price_sqr_foot,housing_units,gross_rent
Alamo Square,37.791012,-122.402100,2013.000000,366.020712,378401.0,2817.285714
Anza Vista,37.779598,-122.443451,2013.333333,373.382198,379050.0,3031.833333
Bayview,37.734670,-122.401060,2012.000000,204.588623,376454.0,2318.400000
Bayview Heights,37.728740,-122.410980,2015.000000,590.792839,382295.0,3739.000000
Bernal Heights,37.728630,-122.443050,NaN,NaN,NaN,NaN


,Lat,Lon,year,sale_price_sqr_foot,housing_units,gross_rent
Yerba Buena,37.79298,-122.39636,2012.5,576.709848,377427.5,2555.166667
Bernal Heights,NaN,NaN,2013.5,576.746488,379374.5,3080.333333
Downtown,NaN,NaN,2013.0,391.434378,378401.0,2817.285714
Ingleside,NaN,NaN,2012.5,367.895144,377427.5,2509.000000
Outer Richmond,NaN,NaN,2013.0,473.900773,378401.0,2817.285714


In [108]:
# Call the dropna function to remove any neighborhoods that do not have data
all_neighborhoods_df = all_neighborhoods_df.reset_index().dropna()

# Rename the "index" column as "Neighborhood" for use in the Visualization
all_neighborhoods_df = all_neighborhoods_df.rename(columns={"index": "Neighborhood"})

# Review the resulting DataFrame
display(all_neighborhoods_df.head())
display(all_neighborhoods_df.tail())

,Neighborhood,Lat,Lon,year,sale_price_sqr_foot,housing_units,gross_rent
0,Alamo Square,37.791012,-122.402100,2013.000000,366.020712,378401.0,2817.285714
1,Anza Vista,37.779598,-122.443451,2013.333333,373.382198,379050.0,3031.833333
2,Bayview,37.734670,-122.401060,2012.000000,204.588623,376454.0,2318.400000
3,Bayview Heights,37.728740,-122.410980,2015.000000,590.792839,382295.0,3739.000000
5,Buena Vista Park,37.768160,-122.439330,2012.833333,452.680591,378076.5,2698.833333


,Neighborhood,Lat,Lon,year,sale_price_sqr_foot,housing_units,gross_rent
68,West Portal,37.74026,-122.463880,2012.25,498.488485,376940.75,2515.500000
69,Western Addition,37.79298,-122.435790,2012.50,307.562201,377427.50,2555.166667
70,Westwood Highlands,37.73470,-122.456854,2012.00,533.703935,376454.00,2250.500000
71,Westwood Park,37.73415,-122.457000,2015.00,687.087575,382295.00,3959.000000
72,Yerba Buena,37.79298,-122.396360,2012.50,576.709848,377427.50,2555.166667


In [111]:
all_neighborhoods_rent_sort = all_neighborhoods_df.sort_values('gross_rent')
all_neighborhoods_sale_price_sort = all_neighborhoods_df.sort_values('sale_price_sqr_foot')
display(all_neighborhoods_rent_sort.tail())
display(all_neighborhoods_sale_price_sort.head())

,Neighborhood,Lat,Lon,year,sale_price_sqr_foot,housing_units,gross_rent
36,Merced Heights,37.71993,-122.46595,2014.0,788.844818,380348.0,3414.0
59,Silver Terrace,37.73467,-122.40106,2014.0,170.292549,380348.0,3528.0
67,Visitacion Valley,37.72874,-122.41098,2014.5,301.466180,381321.5,3657.0
3,Bayview Heights,37.72874,-122.41098,2015.0,590.792839,382295.0,3739.0
71,Westwood Park,37.73415,-122.45700,2015.0,687.087575,382295.0,3959.0


,Neighborhood,Lat,Lon,year,sale_price_sqr_foot,housing_units,gross_rent
59,Silver Terrace,37.73467,-122.40106,2014.0,170.292549,380348.0,3528.00
24,Hunters Point,37.72551,-122.37178,2012.5,170.624920,377427.5,2489.00
2,Bayview,37.73467,-122.40106,2012.0,204.588623,376454.0,2318.40
47,Outer Mission,37.72280,-122.43869,2013.5,242.370952,379374.5,2995.75
67,Visitacion Valley,37.72874,-122.41098,2014.5,301.466180,381321.5,3657.00


### Using hvPlot with GeoViews enabled, create a `points` plot for the `all_neighborhoods_df` DataFrame.

In [110]:
# Create a plot to analyze neighborhood info
all_neighborhoods_df.hvplot.points(
    'Lon',
    'Lat',
    geo=True,
    size='sale_price_sqr_foot',
    color='gross_rent',
    frame_width=700,
    frame_height=500,
    tiles='OSM',
    title='San Francisco Avg. Gross Rent, Avg. Sale Price by Square Foot - 2012-2016',
    fill_alpha=.6,
)

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lon,Lat]   (gross_rent,sale_price_sqr_foot)

### Question Which neighborhood has the highest gross rent, and which has the highest sale price per square foot?
The neighborhood with the highest gross rent was Westwood Park, and the neighborhood with the highest sale price per square foot was Union Square District.

---

## Final Questions
### How does the trend in rental income growth compare to the trend in sales prices? Does this same trend hold true for all the neighborhoods across San Francisco?
The data provided for this analysis indicates that, fairly consistently across all of the neighborhoods surveyed, the average gross rental price has dramatically increased, often by nearly 400%. The data for average sale price per square foot was less consistent, however. Although most neighborhoods saw a fairly significant upward trend in pricing, some seemed to stay the same or even decrease over time.
### What insights can you share with your company about the potential one-click, buy-and-rent strategy that they're pursuing? Do neighborhoods exist that you would suggest for investment, and why?
Given the trends that we can observe from this data, it appears that buying and renting out properties in San Francisco has the potential to be very lucrative. Certain neighborhoods, such as Silver Terrace, Hunters Point, and Bayview seem to offer particularly high opportunities for return given their very high average gross rent price and comparably low average sale price per square foot.

Unfortunately, the question of whether or not this would actually be a good investment strategy is much more complicated and, in my opinion, cannot be answered simply by looking at pricing data. Other factors must be considered in order to make this determination. I would advise investors to proceed with caution when deciding whether to make a purchase, taking into consideration factors such as property tax, rent control, and the possibility of a market downturn. 

On the surface, the one-click, buy-and-rent strategy seems attractive, but it also brings up the question of how much responsibility the company will take for managing liability, such as ensuring that the properties listed on their marketplace are actually viable. For example, since the investor is seeking simplicity with the transaction, that implies that someone else will be handling the process of inspecting the building to ensure that it is fit for habitation. Will the company be responsible for that, or will it be partnering with a third party? The idea of handling such large transactions with one click also becomes particularly problematic when considering where the funds will actually be coming from. I don't think it is possible to employ a one click strategy when the buyer is taking out a loan, and in the case of a cash purchase, how will the company ensure that the buyer is not money laundering?

In conclusion, I think that the idea of a one-click, buy-and-rent real estate investing startup would make for a good sales pitch and would probably be able to generate enough interest to attract investors. However, I think this idea may be a bit too good to be true. When considering how to answer this question, I was reminded of the used car sales company Carvana. The idea - easily buy a car online and have it delivered to your door - seemed so great, but when buyers received their cars, they were unable to get them registered. While I don't think it's *impossible*, I see so much potential for liability and controversy in this strategy that I would not recommend it.